In [5]:
import numpy as np
import pandas as pd

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
from google.colab import files
uploaded =files.upload()
filename = list(uploaded.keys())[0]


Saving credits.csv to credits.csv


In [23]:
filename = 'credits.csv'

In [8]:
metadata = 'movies_metadata.csv'

### 1. 파일 읽어오기(credits) 

In [57]:
credits = pd.read_csv(filename, low_memory=False)
credits.head(2)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


In [58]:
credits.isnull().sum()

cast    0
crew    0
id      0
dtype: int64

In [59]:
movie = pd.read_csv(metadata, low_memory=False)
movie.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [60]:
# movie 데이터에서 필요한 데이터만 추출해서 df 생성 
df = movie[['id', 'title', 'overview']]

In [61]:
# 공백 확인 및 제거 
df.isnull().sum()

id            0
title         6
overview    954
dtype: int64

In [62]:
df.dropna(inplace=True)

In [63]:
# credits 와  movie를 합치기 위해서는 id 기준으로 합쳐야함
# id를 객체에서 정수형으로 변환 

df['id'] = df.id.astype(int)

In [67]:
# credits의 id 도 정수형으로 

credits['id'] = credits.id.astype(int)

#### 데이터 합쳐!! 데이터 전처리

In [69]:
df = df.merge(credits, on='id')
df.set_index('id', inplace=True)
df.head(3)

,title,overview,cast,crew
id,,,,
862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."


In [71]:
df= df.head(20000)
df.tail(3)

,title,overview,cast,crew
id,,,,
70410,The Public Woman,An inexperienced young actress is invited to p...,"[{'cast_id': 2, 'character': 'Ethel', 'credit_...","[{'credit_id': '56cf7c0dc3a3681e4a00370c', 'de..."
64876,Ohyaku: The Female Demon,Ohyaku is a grifter who falls in love with a t...,"[{'cast_id': 1001, 'character': 'Ohyaku Dayu',...","[{'credit_id': '52fe46efc3a368484e0abee5', 'de..."
44470,Ice Cream Man,"Poor Gregory, after being released from the Wi...","[{'cast_id': 2, 'character': 'Ice Cream Man', ...","[{'credit_id': '52fe468ac3a36847f8104907', 'de..."


#### 텍스트 전처리

* overview

In [91]:
df.overview = df.overview.str.replace('[^A-Za-z ]', '')
df.overview[0]

'Led by Woody Andys toys live happily in his room until Andys birthday brings Buzz Lightyear onto the scene Afraid of losing his place in Andys heart Woody plots against Buzz But when circumstances separate Buzz and Woody from their owner the duo eventually learns to put aside their differences'

* 주연배우

In [72]:
# df.cast 의 데이터에서 ' ' 을 제거하는 함수 
from ast import literal_eval
df['cast'] = df.cast.apply(literal_eval)
df.head(3)

,title,overview,cast,crew
id,,,,
862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."


In [75]:
df.cast[826][0]['name']

'William Holden'

In [88]:
# 외국 이름의 경우 "First Name + 빈칸 + Last Name"으로 사용, 
# 텍스트르 합칠 필요가 있음 
import re

def get_3cast(x):
  cast=[]
  for item in x :
    if item['name'] not in cast:
      cast.append(item['name'])
  cast = cast if len(cast) <= 3 else cast[:3]
  cast = list(map(lambda x: re.sub(' ', '',x), cast))
  return ' '.join(cast)

In [89]:
df['cast3'] = df.cast.apply(get_3cast)

In [90]:
df['cast3'][0]

'TomHanks TimAllen DonRickles'

* 감독

In [80]:
df['crew'] = df.crew.apply(literal_eval)
df.head(3)

,title,overview,cast,crew,cast3
id,,,,,
862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",TomHanks
8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",RobinWilliams
15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",WalterMatthau


In [82]:
df.crew[862][0]

{'credit_id': '52fe4284c3a36847f8024f49',
 'department': 'Directing',
 'gender': 2,
 'id': 7879,
 'job': 'Director',
 'name': 'John Lasseter',
 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}

In [83]:
def get_director(x):
  director=[]
  for item in x :
    if item['job'] == 'Director':
      return item['name'].replace(' ', '')
  return ''

In [84]:
df['director'] = df.crew.apply(get_director)

In [91]:
df.reset_index(inplace=True)
df.head(3)

,index,id,title,overview,cast,crew,cast3,director
0,0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",TomHanks TimAllen DonRickles,JohnLasseter
1,1,8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",RobinWilliams JonathanHyde KirstenDunst,JoeJohnston
2,2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",WalterMatthau JackLemmon Ann-Margret,HowardDeutch


## DTM 변환

In [92]:
# 3개를 한번에 수행하기 위해서 total 이라는 column 을 하나 더 만든다

df['total'] = df.overview + ' ' + df.director + ' ' + df.cast3
df.head(3)

,index,id,title,overview,cast,crew,cast3,director,total
0,0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",TomHanks TimAllen DonRickles,JohnLasseter,"Led by Woody, Andy's toys live happily in his ..."
1,1,8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",RobinWilliams JonathanHyde KirstenDunst,JoeJohnston,When siblings Judy and Peter discover an encha...
2,2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",WalterMatthau JackLemmon Ann-Margret,HowardDeutch,A family wedding reignites the ancient feud be...


In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvect = TfidfVectorizer(stop_words='english')
tfidf_matrix = tvect.fit_transform(df.total)
tfidf_matrix.shape

(20000, 76313)

In [94]:
# 영화의 타이틀과 인덱스를 가진 테이블 

indices = pd.Series(df.index, index=df.title).drop_duplicates()
indices.head()

title
Toy Story                      0
Jumanji                        1
Grumpier Old Men               2
Waiting to Exhale              3
Father of the Bride Part II    4
dtype: int64

## 코사인 유사도

In [95]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [96]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df.title.iloc[movie_indices]

In [97]:
get_recommendations('Toy Story')

15347               Toy Story 3
2990                Toy Story 2
10307    The 40 Year Old Virgin
8338                  The Champ
1067      Rebel Without a Cause
1924                  Condorman
11412    For Your Consideration
11618              Factory Girl
17171                 Group Sex
483                      Malice
Name: title, dtype: object